# DBI strategies comparison

This notebook is a comparison of the so-far developed diagonalization strategies for DBI, including the canonical, Pauli-Z, and magnetic field strategies. On top of these, we also show case the use of invariant DBI generators such as 'BHMM'.

In [ ]:
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from qibo import hamiltonians, set_backend
from qibo.hamiltonians import Hamiltonian, SymbolicHamiltonian
from qibo.quantum_info import random_hermitian
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration
from qibo.models.dbi.utils import *

In [ ]:
def visualize_matrix(matrix, title=""):
    """Visualize hamiltonian in a heatmap form."""
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_title(title)
    try:
        im = ax.imshow(np.absolute(matrix), cmap="inferno")
    except TypeError:
        im = ax.imshow(np.absolute(matrix.get()), cmap="inferno")
    fig.colorbar(im, ax=ax)


## Test on random Hamiltonian


In [ ]:
# backend
set_backend("qibojit", "numba")
# initialize dbi object
nqubits = 5
h0 = random_hermitian(2**nqubits, seed=2)
dbi = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0))
print("Initial off diagonal norm", dbi.off_diagonal_norm)
visualize_matrix(dbi.h.matrix, title=f'Random hamiltonian with L={nqubits}')

In [ ]:
# iterations steps
NSTEPS = 15
# choose polynomial scheduling
scheduling = DoubleBracketScheduling.use_hyperopt

### Canonical

In [ ]:
# initialize DBI class for the canonical case
dbi_canonical = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), mode=DoubleBracketGeneratorType.canonical, scheduling=scheduling)

In [ ]:
# Canonical
off_diagonal_norm_history_canonical = [dbi_canonical.off_diagonal_norm]
steps_canonical_plot = [0]
for s in range(NSTEPS):
    # same settings as iteration from list
    step = dbi_canonical.choose_step(d=dbi.diagonal_h_matrix, max_evals=50)
    dbi_canonical(step=step)
    print(f"New optimized step at iteration {s+1}/{NSTEPS}: {step}, loss {dbi_canonical.off_diagonal_norm}")
    off_diagonal_norm_history_canonical.append(dbi_canonical.off_diagonal_norm)
    steps_canonical_plot.append(steps_canonical_plot[-1]+step)

### Pauli-Z

In [ ]:
# initialize DBI class for the Pauli-Z strategy
dbi_pauli = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), mode=DoubleBracketGeneratorType.single_commutator, scheduling=scheduling)

In [ ]:
generate_local_Z = generate_Z_operators(nqubits)
Z_ops = list(generate_local_Z.values())
Z_names = list(generate_local_Z.keys())
Z_optimal = []
# add in initial values for plotting
off_diagonal_norm_history_pauli = [dbi_pauli.off_diagonal_norm]
steps_pauli_plot = [0]
scheduling = DoubleBracketScheduling.use_hyperopt
for _ in range(NSTEPS):
    dbi_pauli, idx, step, flip_sign = select_best_dbr_generator(dbi_pauli, Z_ops, scheduling=scheduling, compare_canonical=False, max_evals=50)
    off_diagonal_norm_history_pauli.append(dbi_pauli.off_diagonal_norm)
    steps_pauli_plot.append(steps_pauli_plot[-1]+step)
    if flip_sign < 0:
        Z_optimal.append('-' + Z_names[idx])
    else:
        Z_optimal.append(Z_names[idx])
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {step} with operator {Z_optimal[-1]}, loss {dbi_pauli.off_diagonal_norm}")

### Magnetic field

In [ ]:
# initialize DBI class for the canonical case
dbi_gradient = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), mode=DoubleBracketGeneratorType.single_commutator, scheduling=scheduling)

In [ ]:
onsite_Z_ops = generate_onsite_Z_ops(nqubits)
d_coef = onsite_Z_decomposition(dbi_gradient.h.matrix, onsite_Z_ops)
d = sum([d_coef[i] * onsite_Z_ops[i] for i in range(nqubits)])

In [ ]:
off_diagonal_norm_history_gradient = [dbi_gradient.off_diagonal_norm]
steps_gradient_plot= [0]
for _ in range(NSTEPS):
    step, d_coef, d = gradient_descent_onsite_Z(dbi_gradient, d_coef, d, onsite_Z_ops=onsite_Z_ops, max_evals=50, n_taylor=5)
    dbi_gradient(d=d,step=step)
    off_diagonal_norm_history_gradient.append(dbi_gradient.off_diagonal_norm)
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {step} with d_coef {d_coef}, loss {dbi_gradient.off_diagonal_norm}")
    steps_gradient_plot.append(steps_gradient_plot[-1]+step)

In [ ]:
plt.title(str(nqubits) + ' random Hamiltonian diagonalization')
plt.plot(off_diagonal_norm_history_canonical, label='canonical')
plt.plot(off_diagonal_norm_history_pauli, label='Pauli-Z')
plt.plot(off_diagonal_norm_history_gradient, label='gradient')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

In [ ]:
plt.title(str(nqubits) + ' random Hamiltonian diagonalization')
plt.plot(steps_canonical_plot, off_diagonal_norm_history_canonical, marker='o', label='canonical')
plt.plot(steps_pauli_plot, off_diagonal_norm_history_pauli, marker='o', label='Pauli-Z')
plt.plot(steps_gradient_plot,off_diagonal_norm_history_gradient, marker='o', label='gradient')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')